In [7]:
# Implement a complete RAG pipeline with a Stuff Documents chain.
# You must implement the chain manually.
# Give a ConversationBufferMemory to the chain.

from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm =ChatOpenAI(temperature=0.1)

#전체 prompt를 메모리에 저장하는 방식 활용
memory = ConversationBufferMemory(
    llm = llm,
    return_messages=True
)

cache_dir = LocalFileStore("../.cache/")

splitter =CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size= 600,
    chunk_overlap= 100,
)
loader = UnstructuredFileLoader("./document.txt")

docs = loader.load_and_split(text_splitter= splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    MessagesPlaceholder(variable_name = "chat_history"),
    ("human","{question}"),
])

def load_memory(input):
    return memory.load_memory_variables({})["history"]

chain = {"context" : retriever, "question": RunnablePassthrough(), "chat_history": load_memory} | prompt | llm

def invoke_chain(question):
    result = chain.invoke(question)   
    memory.save_context({"question": question},{"output": result.content}) 
    print(result)

In [8]:
invoke_chain("Is Aaronson guilty?")

content='According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'


In [9]:
invoke_chain("What message did he write in the table?")

content='He wrote "2+2=5" in the dust on the table.'


In [10]:
invoke_chain("Who is Julia?")

content='Julia is a character who was involved with the protagonist in the story.'
